In [ ]:
import rasterio
import os
from datetime import datetime
import numpy as np

In [ ]:
# local libraries
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model, DFM_model
from coupling_PCR_FM.utils import config_to_dict
%config Application.log_level="INFO"

In [ ]:
# IMPORT MODEL SETTINGS FROM INI-FILE
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# parse set/ini-file with central/general settings for coupling framework
config = config_to_dict(argv1)
# parse env-file for user-specific paths and environmental variables
envs = config_to_dict(argv2)
# combine
config.update(envs)
options = config
# parse dates
start_date = datetime.strptime(options['numerical_settings']['startTime'], '%Y-%m-%d')
end_date = datetime.strptime(options['numerical_settings']['endTime'], '%Y-%m-%d')

In [ ]:
# setup output dir
cwd = os.getcwd() # note: this get changed by pcr initialization later on
out_dir = options['PCRpaths']['outputDirectoryPCR']
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
print(out_dir)
print(cwd)

## create PCR model BMI object

In [ ]:
PCR_config_fn = os.path.join(cwd, options['hydrologic_model']['config_dir'], options['hydrologic_model']['config_file'])
PCR_in_dir = options['PCRpaths']['inputDirectoryPCR']
PCR_out_dir = os.path.join(out_dir, 'PCR')
print(PCR_in_dir)
print(PCR_config_fn)
print(PCR_out_dir)

PCR_bmi = PCR_model(PCR_config_fn, PCR_in_dir, PCR_out_dir,
                            start_date, end_date,dt=1)


## create CMF model BMI object

In [ ]:
CMF_engine = os.path.join(cwd, options['CMF_engine']['CMF_path'])
print(CMF_engine)
CMF_model_dir = os.path.join(cwd, options['routing_model']['model_dir'])
CMF_config_fn = os.path.join(CMF_model_dir, options['routing_model']['model_file'])
CMF_out_dir = os.path.join(out_dir, 'CMF')
     
CMF_bmi = CMF_model(CMF_engine, CMF_config_fn, CMF_model_dir, CMF_out_dir,
                         start_date, end_date, dt=86400)


## create DFM model BMI object

In [ ]:
DFM_engine = os.path.join(cwd, options['DFM_engine']['DFM_path'])
DFM_model_dir = os.path.join(cwd, options['hydrodynamic_model']['model_dir'])
DFM_config_fn = os.path.join(DFM_model_dir, options['hydrodynamic_model']['model_file'])
DFM_out_dir = os.path.join(out_dir, 'DFM')
print(DFM_engine)
print(DFM_model_dir)
print(DFM_config_fn)
print(DFM_out_dir)


DFM_bmi = DFM_model(DFM_engine, DFM_config_fn, DFM_model_dir, DFM_out_dir,
                              start_date, end_date, dt=86400.)

## intitialize models 

In [ ]:
# initialize model at PCR_out_dir
PCR_bmi.initialize()

In [ ]:
# set inpmat based on upstream hydrological model
PCR_bmi.get_model_2d_index()
bounds, res = PCR_bmi.model_grid_bounds, PCR_bmi.model_grid_res
CMF_bmi.set_inpmat_file(bounds, res)

# reset PCR ldd -> no routing
PCR_bmi.deactivate_LDD('all')


In [ ]:
# set update states function
def PCR2CMF_runoff(PCR_bmi, CMF_bmi):
    "creates a 'get_runoff' function to get runoff from PCR model fit for CMF model"
    def update_states():
        "coupling runoff between CMFan PCR model"
        runoff = PCR_bmi.get_var('landSurfaceRunoff')
        runoff = np.where(np.isnan(runoff), 0, runoff)
        CMF_bmi.set_var("runoff", runoff)
    return update_states

update_states = PCR2CMF_runoff(PCR_bmi, CMF_bmi)

# initialize model at CMF_out_dir
CMF_bmi.initialize()

In [ ]:
# initialize DFM to expose model coordinates via BMI
DFM_bmi.initialize()
# get 1d coordinates
DFM_bmi.get_model_coords()

## run coupled PCR - CMF model

In [ ]:
# run coupled model for 20 timesteps
# for i in range(20):
#     PCR_bmi.update()
#     update_states()
#     CMF_bmi.update()   

## set coupling between CMF and DFM

In [ ]:
# get coupled dictionaries
DFM2CMF, CMF2DFM = CMF_bmi.couple_1d_2d(DFM_bmi.model_1d_coords, indices=DFM_bmi.model_1d_indices)

In [ ]:
def CMF2DFM_disch_1d(CMF_bmi, DFM_bmi, DFM2CMF, CMF2DFM):
    ""
    # determine area fraction for volume distribution
    area_frac = {} # [m2/m2]
    area_all = DFM_bmi.get_var('ba') #[m2]
    for CMFidx in CMF2DFM:
        DFMidx = CMF2DFM[CMFidx]
        area_sum = np.sum(area_all[DFMidx])
        afs = {idx: area_all[idx]/area_sum for idx in DFMidx}
        area_frac.update(afs)

    # get short index handles
    DFMidx = DFM_bmi.model_1d_indices
    CMFidx = zip(*[DFM2CMF[idx] for idx in DFMidx]) # tuple of lists

    # sort to DFMidx
    area_frac = np.array([area_frac[idx] for idx in DFMidx])
    area = np.array(area_all[DFMidx])
        
    def update_states():
        discharge_cmf = CMF_bmi.get_var('outflw')[CMFidx] * CMF_bmi.options['dt'] #[m3/dt]
        delta_water_depth_dfm = discharge_cmf * area_frac / area # [m/dt]
        DFM_bmi.set_var_index('rain', DFMidx, delta_water_depth_dfm)
    return update_states

update_states = CMF2DFM_disch_1d(CMF_bmi, DFM_bmi, DFM2CMF, CMF2DFM)

## run coupled CMF - DFM model

In [ ]:
# run coupled model for 2 timesteps
for i in range(2):
    CMF_bmi.update()
    update_states()
    DFM_bmi.update()


## Finalise models

In [ ]:
PCR_bmi.finalize()
CMF_bmi.finalize()
DFM_bmi.finalize()
